This code generates eight different mono fv-1 reverb programs.

Variations:

- Algorithm: plate or room
- Density: sparse or dense
- Pre-delay: short or long

Programs:

- 0 = plate, sparse, short
- 1 = plate, sparse, long
- 2 = plate, dense, short
- 3 = plate, dense, long
- 4 = room, sparse, short
- 5 = room, sparse, long
- 6 = room, dense, short
- 7 = room, dense, long

The pot controls are:

- Pot 0: Reverb decay time
- Pot 1: No connection
- Pot 2: No connection

In [ ]:
import math
import numpy as np
from string import Template

max_sample_time = 32768


We want four times four sets of delay lines. The total time used must be less than 32768.

The plate timings are optimized for maximum smoothness, while the room times are set to provide a more regular reflection pattern characteristic of shoebox shaped rooms. The base times will be different for room and plate algorithms.

The way that this works is that the desired proportions are divided into the max time so that we generate a time unit. The base times are set to multiples of that time unit. Further refinements in tap times will add random numbers to those base units, so don't be scared by nice round numbers here.

The timings in ms are calculated as a handy reference.


In [ ]:
# Define the unit times matrix for the plate
plate_unit_times = np.array([
    [ 2, 7, 10, 15 ],
    [ 3, 6, 11, 14 ],
    [ 4, 9, 12, 17 ],
    [ 5, 8, 13, 16 ]
])

# Derive the time unit
plate_total_time_units = np.sum(plate_unit_times)
print(f"Plate total time units = {plate_total_time_units}")
plate_time_unit = int(max_sample_time/plate_total_time_units)
print(f"Plate time unit = {plate_time_unit}")


# Calculate the base times for each delay
plate_base_times = plate_unit_times * plate_time_unit
print(f"Plate Base times = \n { plate_base_times }")

print(f"Times in ms at 32kHz = \n { plate_base_times * 1/32.768 }")
print(f"Times in ms at 8kHz = \n { plate_base_times * 4/32.768 }")


Plate total time units = 152
Plate time unit = 215
Plate Base times = 
 [[ 430 1505 2150 3225]
 [ 645 1290 2365 3010]
 [ 860 1935 2580 3655]
 [1075 1720 2795 3440]]
Times in ms at 32kHz = 
 [[ 13.12255859  45.92895508  65.61279297  98.41918945]
 [ 19.68383789  39.36767578  72.17407227  91.85791016]
 [ 26.24511719  59.05151367  78.73535156 111.54174805]
 [ 32.80639648  52.49023438  85.29663086 104.98046875]]
Times in ms at 8kHz = 
 [[ 52.49023438 183.71582031 262.45117188 393.67675781]
 [ 78.73535156 157.47070312 288.69628906 367.43164062]
 [104.98046875 236.20605469 314.94140625 446.16699219]
 [131.22558594 209.9609375  341.18652344 419.921875  ]]


In [ ]:
# Define the unit times matrix for the room
room_unit_times = np.array([
    [ 6, 11, 16, 19 ],
    [ 2, 3, 4, 5 ],
    [ 6, 11, 16, 19 ],
    [ 2, 3, 4, 5 ]
])

# Derive the time unit
room_total_time_units = np.sum(room_unit_times)
print(f"Room total time units = {room_total_time_units}")
room_time_unit = int(max_sample_time/room_total_time_units)
print(f"Room time unit = {room_time_unit}")


# Calculate the base times for each delay
room_base_times = room_unit_times * room_time_unit
print(f"Room Base times = \n { room_base_times }")

print(f"Times in ms at 32kHz = \n { room_base_times * 1/32.768 }")
print(f"Times in ms at 8kHz = \n { room_base_times * 4/32.768 }")

Room total time units = 132
Room time unit = 248
Room Base times = 
 [[1488 2728 3968 4712]
 [ 496  744  992 1240]
 [1488 2728 3968 4712]
 [ 496  744  992 1240]]
Times in ms at 32kHz = 
 [[ 45.41015625  83.25195312 121.09375    143.79882812]
 [ 15.13671875  22.70507812  30.2734375   37.84179688]
 [ 45.41015625  83.25195312 121.09375    143.79882812]
 [ 15.13671875  22.70507812  30.2734375   37.84179688]]
Times in ms at 8kHz = 
 [[181.640625  333.0078125 484.375     575.1953125]
 [ 60.546875   90.8203125 121.09375   151.3671875]
 [181.640625  333.0078125 484.375     575.1953125]
 [ 60.546875   90.8203125 121.09375   151.3671875]]


Each program will need 16 random numbers for scattering, all equally spaced within the time unit. (This approach is sometimes called velvet noise.)

Some random combinations may sound better than others, but in general just rolling the dice should be fine.

In [ ]:
def generate_scattering(time_unit):
  velvet_unit = int(time_unit / 16)
  scattering = [(i * velvet_unit + np.random.randint(velvet_unit)) for i in range(16)]
  np.random.shuffle(scattering)
  return np.reshape(scattering, (4,4))

print(f"Example scattering = \n {generate_scattering(plate_time_unit)}")

Example scattering = 
 [[ 20  68 101  51]
 [155 161 134  32]
 [202 181  87 184]
 [107 120  54   8]]


We're going to be using some 1 pole filters. Here is the formula to calculate the coefficient.

In [ ]:
fs = 32000

def onepole_coeff(cutoff):
  wc = cutoff / fs
  b = 2.0 - math.cos(2.0 * math.pi * wc)
  return 1 - (b - math.sqrt(b * b - 1.0))


This is common, boilerplate code that we'll use for all of the algorithms.



In [ ]:
# Memory declarations for the plate algorithms

plate_base_time_lines = ""
for i in range(4):
  for j in range(4):
    plate_base_time_lines += f"  MEM d{i}{j} {plate_base_times[i, j]}\n"

# Memory declarations for the room algorithms

room_base_time_lines = ""
for i in range(4):
  for j in range(4):
    room_base_time_lines += f"  MEM d{i}{j} {room_base_times[i, j]}\n"

common_header = f"""
; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  {onepole_coeff(1000)}
  EQU k_2kHz  {onepole_coeff(2000)}
  EQU k_4kHz  {onepole_coeff(4000)}
  EQU k_8kHz  {onepole_coeff(8000)}

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  WRAX rt, 0.0

; Sum the left and right inputs.
; Leave some headroom for processing.

adc_input:
  RDAX adcl, 0.25
  RDAX adcr, 0.25
  WRAX input, 0.0

"""

Most of the processing is defined by the following lattice structure. The reverb topology is a set of four, four channel FDNs which are organized in a circular loop.

- d0x -> d1x -> d2x -> d3x -^

There are two main variations, sparse and dense. The sparse variant is a traditional FDN in which the four delay lines are mixed with a Hadamard matrix. The dense variant collapses two FDN operations into a single matrix multiplication and produces exponentially more reflection mixing than the sparse variation.

In [ ]:
class Lattice:
    """
    Four delay lines + a hadamard mixing matrix + some tricks

    Attributes:
        name (str): A unique name for the lattice (e.g., "d0x").
        delay_names (list): A list of strings representing the names of
                            the delay memory elements used by this lattice.
        out_delay_names (list): A list of strings representing the names of
                                the delay memory elements where the output
                                of this lattice is written.
        dtimes (list): A list of integers representing the diffusion
                                times (taps) for the lattice. These are typically
                                prime numbers.
        scattering (string): 'sparse' or 'dense'
        in_channel (int): The input channel index (0, 1, 2, or 3) where
                          the input signal is added to the lattice calculation.
        lpf (string): Whether to insert a lpf in the feedback path,
                      string should be set to the equate with the coefficient.
    """
    def __init__(self, name, delay_names, out_delay_names, dtimes, scattering='sparse', in_channel=None, lpf=None):
        self.name = name
        self.dn = delay_names
        self.odn = out_delay_names
        self.inc = in_channel
        self.input_reg = "input"
        self.lpf = lpf
        self.scattering = scattering
        self.dt = np.tile(dtimes, (4,1));
        if scattering == 'dense':
          self.dt = self.dt.T # Magic


    def print(self):
        print(f"calc_lattice_{self.scattering}_{self.name}:\n")
        print(f"  RDA {self.dn[0]}# - {self.dt[0, 0]}, 0.5")
        print(f"  RDA {self.dn[1]}# - {self.dt[0, 1]}, 0.5")
        print(f"  RDA {self.dn[2]}# - {self.dt[0, 2]}, 0.5")
        print(f"  RDA {self.dn[3]}# - {self.dt[0, 3]}, 0.5")
        print(f"  WRAX out_{self.name}, 1.0")
        print(f"  MULX rt")
        if self.inc == 0:
          print(f"  RDAX {self.input_reg}, 1.0")
        print(f"  WRA {self.odn[0]}, 0.0\n")

        print(f"  RDA {self.dn[0]}# - {self.dt[1, 0]}, 0.5")
        print(f"  RDA {self.dn[1]}# - {self.dt[1, 1]}, -0.5")
        print(f"  RDA {self.dn[2]}# - {self.dt[1, 2]}, 0.5")
        print(f"  RDA {self.dn[3]}# - {self.dt[1, 3]}, -0.5")
        print(f"  MULX rt")
        if self.inc == 1:
          print(f"  RDAX {self.input_reg}, 1.0")
        print(f"  WRA {self.odn[1]}, 0.0\n")

        print(f"  RDA {self.dn[0]}# - {self.dt[2, 0]}, 0.5")
        print(f"  RDA {self.dn[1]}# - {self.dt[2, 1]}, 0.5")
        print(f"  RDA {self.dn[2]}# - {self.dt[2, 2]}, -0.5")
        print(f"  RDA {self.dn[3]}# - {self.dt[2, 3]}, -0.5")
        print(f"  MULX rt")
        if self.lpf != None:
          print(f"  RDFX lpf_{self.name}, {self.lpf}")
          print(f"  WRAX lpf_{self.name}, 1.0")
        if self.inc == 2:
          print(f"  RDAX {self.input_reg}, 1.0")
        print(f"  WRA {self.odn[2]}, 0.0\n")

        print(f"  RDA {self.dn[0]}# - {self.dt[3, 0]}, 0.5")
        print(f"  RDA {self.dn[1]}# - {self.dt[3, 1]}, -0.5")
        print(f"  RDA {self.dn[2]}# - {self.dt[3, 2]}, -0.5")
        print(f"  RDA {self.dn[3]}# - {self.dt[3, 3]}, 0.5")
        print(f"  MULX rt")
        if self.inc == 3:
          print(f"  RDAX {self.input_reg}, 1.0")
        print(f"  WRA {self.odn[3]}, 0.0\n")


In [ ]:
# The sparse plate programs will have two input signals and mix two of the matrix outputs.
# Some make up gain in necessary to restore the original signal level for output.
# There is a waveshaper to provide soft clipping on the output
# and it is set up to be a little more aggressive on the sparse output.

sparse_plate_output = f"""
sparse_output:

  RDAX out_d0x, 1.0
  RDAX out_d1x, 1.0
  SOF -1.5, 0.0
  SOF -1.5, 0.0
  WRAX out, 1.0

output_shaper:

  WRAX lim, -0.33333
  MULX lim
  MULX lim
  RDAX lim, 1.0
  SOF -1.5, 0.0
  RDAX out, 0.5

  WRAX dacl, 1.0
  WRAX dacr, 0.0  ; Remove for production
"""

# The dense programs will have four input signals and mix four of the matrix outputs.
# The output taps are all the same level to make the reverb envelope as even as possible.
# The output shaper is less aggressive.

dense_plate_output = f"""
dense_output:

  RDAX out_d0x, 1.0
  RDAX out_d1x, 1.0
  RDAX out_d2x, 1.0
  RDAX out_d3x, 1.0
  SOF -1.5, 0.0
  SOF -1.5, 0.0
  WRAX out, 1.0


output_shaper:

  WRAX lim, -0.33333
  MULX lim
  MULX lim
  RDAX lim, 1.0
  RDAX out, 1.0

  WRAX dacl, 1.0
  WRAX dacr, 0.0   ; Remove for production
"""

# Currently writing to both left and right outputs for ease of testing,
# but the final WRAX can be removed if an additional instruction is needed.

## Plate algorithms

In [ ]:
program = 0
algorithm = "plate"
density = "sparse"
predelay = "short"

scattering = generate_scattering(plate_time_unit)

plate_d0x = Lattice(
    "d0x",
    delay_names = ["d00", "d01", "d02", "d03"],
    out_delay_names = ["d10", "d11", "d12", "d13"],
    dtimes = scattering[0],
    scattering = 'sparse',
    in_channel = None,
    lpf = None)

plate_d1x = Lattice(
    "d1x",
    delay_names = ["d10", "d11", "d12", "d13"],
    out_delay_names = ["d20", "d21", "d22", "d23"],
    dtimes = scattering[1],
    scattering = 'sparse',
    in_channel = None,
    lpf = None)

plate_d2x = Lattice(
    "d2x",
    delay_names = ["d20", "d21", "d22", "d23"],
    out_delay_names = ["d30", "d31", "d32", "d33"],
    dtimes = scattering[2],
    scattering = 'sparse',
    in_channel = 0,
    lpf = None)

plate_d3x = Lattice(
    "d3x",
    delay_names = ["d30", "d31", "d32", "d33"],
    out_delay_names = ["d00", "d01", "d02", "d03"],
    dtimes = scattering[3],
    scattering = 'sparse',
    in_channel = 0,
    lpf = "k_8kHz")

print(f"; Program {program}")
print(f"; Algorithm: {algorithm}, Density: {density}, Pre-delay: {predelay} \n")
print(plate_base_time_lines)
print(common_header)
plate_d0x.print()
plate_d1x.print()
plate_d2x.print()
plate_d3x.print()
print(sparse_plate_output)

; Program 0
; Algorithm: plate, Density: sparse, Pre-delay: short 

  MEM d00 430
  MEM d01 1505
  MEM d02 2150
  MEM d03 3225
  MEM d10 645
  MEM d11 1290
  MEM d12 2365
  MEM d13 3010
  MEM d20 860
  MEM d21 1935
  MEM d22 2580
  MEM d23 3655
  MEM d30 1075
  MEM d31 1720
  MEM d32 2795
  MEM d33 3440


; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  0.17775899601706946
  EQU k_2kHz  0.3214160220919626
  EQU k_4kHz  0.5266022816341156
  EQU k_8kHz  0.7320508075688772

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  

In [ ]:
program = 1
algorithm = "plate"
density = "sparse"
predelay = "long"

scattering = generate_scattering(plate_time_unit)

plate_d0x = Lattice(
    "d0x",
    delay_names = ["d00", "d01", "d02", "d03"],
    out_delay_names = ["d10", "d11", "d12", "d13"],
    dtimes = scattering[0],
    scattering = 'sparse',
    in_channel = None,
    lpf = None)

plate_d1x = Lattice(
    "d1x",
    delay_names = ["d10", "d11", "d12", "d13"],
    out_delay_names = ["d20", "d21", "d22", "d23"],
    dtimes = scattering[1],
    scattering = 'sparse',
    in_channel = None,
    lpf = None)

plate_d2x = Lattice(
    "d2x",
    delay_names = ["d20", "d21", "d22", "d23"],
    out_delay_names = ["d30", "d31", "d32", "d33"],
    dtimes = scattering[2],
    scattering = 'sparse',
    in_channel = 2,
    lpf = None)

plate_d3x = Lattice(
    "d3x",
    delay_names = ["d30", "d31", "d32", "d33"],
    out_delay_names = ["d00", "d01", "d02", "d03"],
    dtimes = scattering[3],
    scattering = 'sparse',
    in_channel = 3,
    lpf = "k_8kHz")

print(f"; Program {program}")
print(f"; Algorithm: {algorithm}, Density: {density}, Pre-delay: {predelay} \n")
print(plate_base_time_lines)
print(common_header)
plate_d0x.print()
plate_d1x.print()
plate_d2x.print()
plate_d3x.print()
print(sparse_plate_output)

; Program 1
; Algorithm: plate, Density: sparse, Pre-delay: long 

  MEM d00 430
  MEM d01 1505
  MEM d02 2150
  MEM d03 3225
  MEM d10 645
  MEM d11 1290
  MEM d12 2365
  MEM d13 3010
  MEM d20 860
  MEM d21 1935
  MEM d22 2580
  MEM d23 3655
  MEM d30 1075
  MEM d31 1720
  MEM d32 2795
  MEM d33 3440


; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  0.17775899601706946
  EQU k_2kHz  0.3214160220919626
  EQU k_4kHz  0.5266022816341156
  EQU k_8kHz  0.7320508075688772

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  W

In [ ]:
program = 2
algorithm = "plate"
density = "dense"
predelay = "short"

scattering = generate_scattering(plate_time_unit)

plate_d0x = Lattice(
    "d0x",
    delay_names = ["d00", "d01", "d02", "d03"],
    out_delay_names = ["d10", "d11", "d12", "d13"],
    dtimes = scattering[0],
    scattering = 'dense',
    in_channel = 0,
    lpf = None)

plate_d1x = Lattice(
    "d1x",
    delay_names = ["d10", "d11", "d12", "d13"],
    out_delay_names = ["d20", "d21", "d22", "d23"],
    dtimes = scattering[1],
    scattering = 'dense',
    in_channel = 0,
    lpf = None)

plate_d2x = Lattice(
    "d2x",
    delay_names = ["d20", "d21", "d22", "d23"],
    out_delay_names = ["d30", "d31", "d32", "d33"],
    dtimes = scattering[2],
    scattering = 'dense',
    in_channel = 0,
    lpf = None)

plate_d3x = Lattice(
    "d3x",
    delay_names = ["d30", "d31", "d32", "d33"],
    out_delay_names = ["d00", "d01", "d02", "d03"],
    dtimes = scattering[3],
    scattering = 'dense',
    in_channel = 0,
    lpf = "k_8kHz")

print(f"; Program {program}")
print(f"; Algorithm: {algorithm}, Density: {density}, Pre-delay: {predelay} \n")
print(plate_base_time_lines)
print(common_header)
plate_d0x.print()
plate_d1x.print()
plate_d2x.print()
plate_d3x.print()
print(dense_plate_output)

; Program 2
; Algorithm: plate, Density: dense, Pre-delay: short 

  MEM d00 430
  MEM d01 1505
  MEM d02 2150
  MEM d03 3225
  MEM d10 645
  MEM d11 1290
  MEM d12 2365
  MEM d13 3010
  MEM d20 860
  MEM d21 1935
  MEM d22 2580
  MEM d23 3655
  MEM d30 1075
  MEM d31 1720
  MEM d32 2795
  MEM d33 3440


; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  0.17775899601706946
  EQU k_2kHz  0.3214160220919626
  EQU k_4kHz  0.5266022816341156
  EQU k_8kHz  0.7320508075688772

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  W

In [ ]:
program = 3
algorithm = "plate"
density = "dense"
predelay = "long"

scattering = generate_scattering(plate_time_unit)

plate_d0x = Lattice(
    "d0x",
    delay_names = ["d00", "d01", "d02", "d03"],
    out_delay_names = ["d10", "d11", "d12", "d13"],
    dtimes = scattering[0],
    scattering = 'dense',
    in_channel = 2,
    lpf = None)

plate_d1x = Lattice(
    "d1x",
    delay_names = ["d10", "d11", "d12", "d13"],
    out_delay_names = ["d20", "d21", "d22", "d23"],
    dtimes = scattering[1],
    scattering = 'dense',
    in_channel = 3,
    lpf = None)

plate_d2x = Lattice(
    "d2x",
    delay_names = ["d20", "d21", "d22", "d23"],
    out_delay_names = ["d30", "d31", "d32", "d33"],
    dtimes = scattering[2],
    scattering = 'dense',
    in_channel = 2,
    lpf = None)

plate_d3x = Lattice(
    "d3x",
    delay_names = ["d30", "d31", "d32", "d33"],
    out_delay_names = ["d00", "d01", "d02", "d03"],
    dtimes = scattering[3],
    scattering = 'dense',
    in_channel = 3,
    lpf = "k_8kHz")

print(f"; Program {program}")
print(f"; Algorithm: {algorithm}, Density: {density}, Predelay: {predelay} \n")
print(plate_base_time_lines)
print(common_header)
plate_d0x.print()
plate_d1x.print()
plate_d2x.print()
plate_d3x.print()
print(dense_plate_output)

; Program 3
; Algorithm: plate, Density: dense, Predelay: long 

  MEM d00 430
  MEM d01 1505
  MEM d02 2150
  MEM d03 3225
  MEM d10 645
  MEM d11 1290
  MEM d12 2365
  MEM d13 3010
  MEM d20 860
  MEM d21 1935
  MEM d22 2580
  MEM d23 3655
  MEM d30 1075
  MEM d31 1720
  MEM d32 2795
  MEM d33 3440


; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  0.17775899601706946
  EQU k_2kHz  0.3214160220919626
  EQU k_4kHz  0.5266022816341156
  EQU k_8kHz  0.7320508075688772

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  WRA

In [ ]:
# The sparse room programs will have two input signals and mix two of the matrix outputs.
# Some make up gain in necessary to restore the original signal level for output.
# There is a waveshaper to provide soft clipping on the output.

sparse_room_output = f"""
sparse_output:

  RDAX out_d0x, 1.0
  RDAX out_d3x, 1.0
  SOF -1.5, 0.0
  SOF -1.5, 0.0
  WRAX out, 1.0

output_shaper:

  WRAX lim, -0.33333
  MULX lim
  MULX lim
  RDAX lim, 1.0
  SOF -1.5, 0.0
  RDAX out, 0.5

  WRAX dacl, 1.0
  WRAX dacr, 0.0  ; Remove for production
"""

# The dense programs will have four input signals and mix four of the matrix outputs.
# The levels of the output taps are set to give the output a nice envelope.
# Some make up gain in necessary to restore the original signal level for output.
# The waveshaper is less aggressive than in the sparse output.

dense_room_output = f"""
dense_output:

  RDAX out_d0x, 0.25
  RDAX out_d1x, 1.0
  RDAX out_d2x, 0.5
  RDAX out_d3x, 1.0
  SOF -2.0, 0.0
  SOF -2.0, 0.0
  WRAX out, 1.0

output_shaper:

  WRAX lim, -0.33333
  MULX lim
  MULX lim
  RDAX lim, 1.0
  RDAX out, 1.0

  WRAX dacl, 1.0
  WRAX dacr, 0.0   ; Remove for production
"""

# Currently writing to both left and right outputs for ease of testing,
# but the final WRAX can be removed if an additional instruction is needed.

## Room Algorithms


In [ ]:
program = 4
algorithm = "room"
density = "sparse"
predelay = "short"

scattering = generate_scattering(room_time_unit)

room_d0x = Lattice(
    "d0x",
    delay_names = ["d00", "d01", "d02", "d03"],
    out_delay_names = ["d10", "d11", "d12", "d13"],
    dtimes = scattering[0],
    scattering = 'sparse',
    in_channel = None,
    lpf = None)

room_d1x = Lattice(
    "d1x",
    delay_names = ["d10", "d11", "d12", "d13"],
    out_delay_names = ["d20", "d21", "d22", "d23"],
    dtimes = scattering[1],
    scattering = 'sparse',
    in_channel = 0,
    lpf = "k_4kHz")

room_d2x = Lattice(
    "d2x",
    delay_names = ["d20", "d21", "d22", "d23"],
    out_delay_names = ["d30", "d31", "d32", "d33"],
    dtimes = scattering[2],
    scattering = 'sparse',
    in_channel = None,
    lpf = None)

room_d3x = Lattice(
    "d3x",
    delay_names = ["d30", "d31", "d32", "d33"],
    out_delay_names = ["d00", "d01", "d02", "d03"],
    dtimes = scattering[3],
    scattering = 'sparse',
    in_channel = 0,
    lpf = "k_8kHz")

print(f"; Program {program}")
print(f"; Algorithm: {algorithm}, Density: {density}, Pre-delay: {predelay} \n")
print(room_base_time_lines)
print(common_header)
room_d0x.print()
room_d1x.print()
room_d2x.print()
room_d3x.print()
print(sparse_room_output)

; Program 4
; Algorithm: room, Density: sparse, Pre-delay: short 

  MEM d00 1488
  MEM d01 2728
  MEM d02 3968
  MEM d03 4712
  MEM d10 496
  MEM d11 744
  MEM d12 992
  MEM d13 1240
  MEM d20 1488
  MEM d21 2728
  MEM d22 3968
  MEM d23 4712
  MEM d30 496
  MEM d31 744
  MEM d32 992
  MEM d33 1240


; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  0.17775899601706946
  EQU k_2kHz  0.3214160220919626
  EQU k_4kHz  0.5266022816341156
  EQU k_8kHz  0.7320508075688772

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  WRAX

In [ ]:
program = 5
algorithm = "room"
density = "sparse"
predelay = "long"

scattering = generate_scattering(room_time_unit)

room_d0x = Lattice(
    "d0x",
    delay_names = ["d00", "d01", "d02", "d03"],
    out_delay_names = ["d10", "d11", "d12", "d13"],
    dtimes = scattering[0],
    scattering = 'sparse',
    in_channel = None,
    lpf = None)

room_d1x = Lattice(
    "d1x",
    delay_names = ["d10", "d11", "d12", "d13"],
    out_delay_names = ["d20", "d21", "d22", "d23"],
    dtimes = scattering[1],
    scattering = 'sparse',
    in_channel = 3,
    lpf = "k_4kHz")

room_d2x = Lattice(
    "d2x",
    delay_names = ["d20", "d21", "d22", "d23"],
    out_delay_names = ["d30", "d31", "d32", "d33"],
    dtimes = scattering[2],
    scattering = 'sparse',
    in_channel = None,
    lpf = None)

room_d3x = Lattice(
    "d3x",
    delay_names = ["d30", "d31", "d32", "d33"],
    out_delay_names = ["d00", "d01", "d02", "d03"],
    dtimes = scattering[3],
    scattering = 'sparse',
    in_channel = 3,
    lpf = "k_8kHz")

print(f"; Program {program}")
print(f"; Algorithm: {algorithm}, Density: {density}, Pre-delay: {predelay} \n")
print(room_base_time_lines)
print(common_header)
room_d0x.print()
room_d1x.print()
room_d2x.print()
room_d3x.print()
print(sparse_room_output)

; Program 5
; Algorithm: room, Density: sparse, Pre-delay: long 

  MEM d00 1488
  MEM d01 2728
  MEM d02 3968
  MEM d03 4712
  MEM d10 496
  MEM d11 744
  MEM d12 992
  MEM d13 1240
  MEM d20 1488
  MEM d21 2728
  MEM d22 3968
  MEM d23 4712
  MEM d30 496
  MEM d31 744
  MEM d32 992
  MEM d33 1240


; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  0.17775899601706946
  EQU k_2kHz  0.3214160220919626
  EQU k_4kHz  0.5266022816341156
  EQU k_8kHz  0.7320508075688772

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  WRAX 

In [ ]:
program = 6
algorithm = "room"
density = "dense"
predelay = "short"

scattering = generate_scattering(room_time_unit)

room_d0x = Lattice(
    "d0x",
    delay_names = ["d00", "d01", "d02", "d03"],
    out_delay_names = ["d10", "d11", "d12", "d13"],
    dtimes = scattering[0],
    scattering = 'dense',
    in_channel = None,
    lpf = None)

room_d1x = Lattice(
    "d1x",
    delay_names = ["d10", "d11", "d12", "d13"],
    out_delay_names = ["d20", "d21", "d22", "d23"],
    dtimes = scattering[1],
    scattering = 'dense',
    in_channel = 0,
    lpf = "k_4kHz")

room_d2x = Lattice(
    "d2x",
    delay_names = ["d20", "d21", "d22", "d23"],
    out_delay_names = ["d30", "d31", "d32", "d33"],
    dtimes = scattering[2],
    scattering = 'dense',
    in_channel = None,
    lpf = None)

room_d3x = Lattice(
    "d3x",
    delay_names = ["d30", "d31", "d32", "d33"],
    out_delay_names = ["d00", "d01", "d02", "d03"],
    dtimes = scattering[3],
    scattering = 'dense',
    in_channel = 0,
    lpf = "k_8kHz")

print(f"; Program {program}")
print(f"; Algorithm: {algorithm}, Density: {density}, Pre-delay: {predelay} \n")
print(room_base_time_lines)
print(common_header)
room_d0x.print()
room_d1x.print()
room_d2x.print()
room_d3x.print()
print(dense_room_output)

; Program 6
; Algorithm: room, Density: dense, Pre-delay: short 

  MEM d00 1488
  MEM d01 2728
  MEM d02 3968
  MEM d03 4712
  MEM d10 496
  MEM d11 744
  MEM d12 992
  MEM d13 1240
  MEM d20 1488
  MEM d21 2728
  MEM d22 3968
  MEM d23 4712
  MEM d30 496
  MEM d31 744
  MEM d32 992
  MEM d33 1240


; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  0.17775899601706946
  EQU k_2kHz  0.3214160220919626
  EQU k_4kHz  0.5266022816341156
  EQU k_8kHz  0.7320508075688772

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  WRAX 

In [ ]:
program = 7
algorithm = "room"
density = "dense"
predelay = "long"

scattering = generate_scattering(room_time_unit)

room_d0x = Lattice(
    "d0x",
    delay_names = ["d00", "d01", "d02", "d03"],
    out_delay_names = ["d10", "d11", "d12", "d13"],
    dtimes = scattering[0],
    scattering = 'dense',
    in_channel = None,
    lpf = None)

room_d1x = Lattice(
    "d1x",
    delay_names = ["d10", "d11", "d12", "d13"],
    out_delay_names = ["d20", "d21", "d22", "d23"],
    dtimes = scattering[1],
    scattering = 'dense',
    in_channel = 3,
    lpf = "k_4kHz")

room_d2x = Lattice(
    "d2x",
    delay_names = ["d20", "d21", "d22", "d23"],
    out_delay_names = ["d30", "d31", "d32", "d33"],
    dtimes = scattering[2],
    scattering = 'dense',
    in_channel = None,
    lpf = None)

room_d3x = Lattice(
    "d3x",
    delay_names = ["d30", "d31", "d32", "d33"],
    out_delay_names = ["d00", "d01", "d02", "d03"],
    dtimes = scattering[3],
    scattering = 'dense',
    in_channel = 2,
    lpf = "k_8kHz")

print(f"; Program {program}")
print(f"; Algorithm: {algorithm}, Density: {density}, Pre-delay: {predelay} \n")
print(room_base_time_lines)
print(common_header)
room_d0x.print()
room_d1x.print()
room_d2x.print()
room_d3x.print()
print(dense_room_output)

; Program 7
; Algorithm: room, Density: dense, Pre-delay: long 

  MEM d00 1488
  MEM d01 2728
  MEM d02 3968
  MEM d03 4712
  MEM d10 496
  MEM d11 744
  MEM d12 992
  MEM d13 1240
  MEM d20 1488
  MEM d21 2728
  MEM d22 3968
  MEM d23 4712
  MEM d30 496
  MEM d31 744
  MEM d32 992
  MEM d33 1240


; Registers

  EQU input reg1; Signal input
  EQU rt reg2 ; Loss factor that sets reverb time

  EQU lpf_d0x reg3 ; Lattice lowpass filters
  EQU lpf_d1x reg4
  EQU lpf_d2x reg5
  EQU lpf_d3x reg6

  EQU out_d0x reg7 ; Lattice outputs
  EQU out_d1x reg8
  EQU out_d2x reg9
  EQU out_d3x reg10

  EQU out ; Output signal
  EQU lim ; Output limiter/shaper

; Constants
; Filter coefficients are for 32kHz

  EQU k_1kHz  0.17775899601706946
  EQU k_2kHz  0.3214160220919626
  EQU k_4kHz  0.5266022816341156
  EQU k_8kHz  0.7320508075688772

; Pot0 controls the reverb time.
; Square root the pot to bias it upwards into a more useful range.

rt_pot:
  RDAX pot0, 0.99
  LOG 0.5, 0
  EXP 1.0, 0
  WRAX r

Aside: all of these programs can be trivially extended to stereo operation by adjusting the input and output taps to inject left into the first two lattices and right to the latter two. Spread output taps as desired.